In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import numpy as np
import pretty_midi
import visual_midi
import joblib
from os import listdir
from os.path import getsize
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import Normalizer, MinMaxScaler

### Get data

In [3]:
path = '../raw_data/pretty_midi'
directory = [file_name for file_name in listdir(path) if getsize(f'{path}/{file_name}') < 350_000]

for i, file_name in enumerate(directory):
    file = joblib.load(f'{path}/{file_name}')
    if i == 0:
        X, y = create_simple_dataset(file, ratio=0.2)
    else:
        try:
            loaded = create_simple_dataset(file, ratio=0.2)
            X = np.concatenate((X, loaded[0]))
            y = np.concatenate((y, loaded[1]))
            del([loaded, file])
        except:
            pass
    if i % 10 == 0:
        print(f'{X.shape[0]} observations')
    if X.shape[0] >= 3_500:
        break

print('Loading chord feature')

chord = adding_chords_info('../raw_data/chords_midi.csv', X, verbose=1)

print('Chord feature loaded')

X = X.reshape((X.shape[0], -1))
X = np.concatenate((chord, X), axis=1, dtype=np.int8)

print('Chord feature implemented')

y = y.reshape((y.shape[0], -1))

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=2)

print('Done')

14 observations
119 observations
232 observations
355 observations
503 observations
640 observations
742 observations
857 observations
973 observations
1102 observations
1209 observations
1321 observations
1435 observations
1589 observations
1750 observations
1861 observations
2060 observations
2168 observations
2280 observations
2390 observations
2516 observations
2655 observations
2808 observations
2952 observations
3048 observations
3164 observations
3312 observations
3393 observations
3489 observations
Loading chord feature
1 done
101 done
201 done
301 done
401 done
501 done
601 done
701 done
801 done
901 done
1001 done
1101 done
1201 done
1301 done
1401 done
1501 done
1601 done
1701 done
1801 done
1901 done
2001 done
2101 done
2201 done
2301 done
2401 done
2501 done
2601 done
2701 done
2801 done
2901 done
3001 done
3101 done
3201 done
3301 done
3401 done
3501 done
Chord feature loaded
Chord feature implemented
Done


### Make predictions

#### Initiate, train and save our model

In [4]:
# Latest best params

best_params = {
    'criterion': 'friedman_mse',
    # 'max_depth': None,
    # 'min_samples_split': 2,
    # 'min_samples_leaf': 1,
    # 'min_weight_fraction_leaf': 0.0,
    # 'max_leaf_nodes': 128,
    'min_impurity_decrease': 0.5
}

In [5]:
# New best params

# best_params = {
#     # 'criterion': 'absolute_error',
#     'max_depth': 12,
#     'min_samples_split': 5,
#     'min_samples_leaf': 1,
#     'min_weight_fraction_leaf': 0.0,
#     'max_leaf_nodes': 12,
#     'min_impurity_decrease':0.5
#}

In [6]:
# Old best params

# best_params = {
#     'criterion': 'friedman_mse',
#     'max_depth': None,
#     'min_samples_split': 3,
#     'min_samples_leaf': 1,
#     'min_weight_fraction_leaf': 0.0,
#     'max_leaf_nodes': 128,
#     'min_impurity_decrease': 0.8
# }

In [4]:
tree = DecisionTreeRegressor()
tree.fit(X, y)
joblib.dump(tree, '../Models/API_tree.joblib')
# tree = joblib.load('../Models/tree.joblib')

['../Models/API_tree.joblib']

#### Attempt to debug it

In [8]:
# example_file = []
# for file in directory:
#     if 200_000 > getsize(f'{path}/{file}'): # We don't want too big or too little files
#         example_file.append(joblib.load(f'{path}/{file}'))
#         break

In [9]:
# dir = 'first_pres_dataset'
# for i, file in enumerate(example_file):
#     create_nparray_dataset(file, dir, name=f'chose{i}')

In [10]:
# X_test, y_test = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/chose0')

In [11]:
# X_test, y_test = create_nparray_dataset(example_file[0], 'f', 'n', store=False)

In [12]:
# X_test = X_test.reshape((X_test.shape[0], -1))
# y_test = y_test.reshape((y_test.shape[0], -1))

#### Predictions

In [17]:
# Making our predictions
# X_test_scale = scaler.transform(X_test)
predictions = tree.predict(X_test)

NameError: name 'X_test' is not defined

In [9]:
# p = pretty_midi.PrettyMIDI('pers.mid').get_piano_roll(fs=50)

In [11]:
def reshape_piano_roll(piano_roll, size=500):
    """
    Reshape the piano roll to match into the required shape for predictions
    """
    if piano_roll.shape[-1] < size:
        
        zeros_array = np.zeros((128, size - piano_roll.shape[-1]))
        return np.concatenate((piano_roll, zeros_array), axis=1)
    
    if piano_roll.shape[-1] > size:
        
        piano_rolls =  np.split(piano_roll, [500], axis=1)
        
        while piano_rolls[-1].shape[-1] > 500:
            
            last_split = np.split(piano_rolls[-1], [500], axis=1)
            piano_rolls[-1] = last_split[0]
            piano_rolls.append(last_split[-1])
        
        # while np.array(piano_rolls[-1]).shape > size:
        #     piano_rolls.append(np.split(piano_roll[-1], [500], axis=1))
        return piano_rolls

In [12]:
# p = reshape_piano_roll(p)

In [14]:
# p = p.reshape(1, 128, 500)

In [15]:
# chord = adding_chords_info('../raw_data/chords_midi.csv', p)

In [16]:
# p_reshaped = p.reshape(1, -1)
# p_reshaped = np.concatenate((chord, p_reshaped))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 24 and the array at index 1 has size 64000

In [ ]:
# predictions = tree.predict(p)

In [14]:
# Test for the "false predictions" problem
for i, pred in enumerate(predictions):
    if i % 10 == 0:
        print((pred == y_test[i]).mean())

0.5


In [15]:
predicted_melodies = np.array([assembled_target_to_melody(prediction) for prediction in predictions])
true_melodies = np.array([assembled_target_to_melody(melody) for melody in y_test])

In [16]:
# They should be different

In [17]:
X_test = np.delete(X_test, np.s_[0:24], axis=1)
X_test = X_test.reshape(-1, 128, 500)

In [18]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(X_test[1]))

Loading BokehJS ...

Column(id='1106', ...)

In [19]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(predicted_melodies[1]))

Loading BokehJS ...

Column(id='1284', ...)

In [20]:
piano_roll_to_pretty_midi(predicted_melodies[1]).write('test.mid')

In [21]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(true_melodies[1]))

Loading BokehJS ...

Column(id='1459', ...)

In [22]:
copy = X_test.copy()
copy_2 = X_test.copy()

In [23]:
assembled_predicted_music = np.array(
    [assemblate_accompaniment_melody(test, predicted_melody) for test, predicted_melody in zip(copy, predicted_melodies)],
)

assembled_true_music = np.array(
    [assemblate_accompaniment_melody(test, true_melody) for test, true_melody in zip(copy_2, true_melodies)],
)

In [24]:
for i, (pred, true) in enumerate(zip(assembled_predicted_music, assembled_true_music)):
    try:
        piano_roll_to_pretty_midi(pred, fs=50).write(f'../PredVSTrue/ManualTuning/{i+4}_pred.mid')
        piano_roll_to_pretty_midi(true, fs=50).write(f'../PredVSTrue/ManualTuning/{i+4}_true.mid')
    except:
        pass